This notebook is a textbook for A tutorial on spectral clustering, I set some examples to help understand the therom.

# similarity graphs

1. ε-neighborhood
2. k-nearest neighbor graphs
3. the fully connected graph 

# Graph Laplacians and their basic properties

assume: G is an undirected, weighted graph with weight matrix W. Eigenvectors of a matrix don't have to be normalized.

## The unnormalized graph Laplacian

L = D - W 

we have the proposition of L


In [10]:
import numpy as np
from numpy.linalg import eigh
import sympy as sp

# assume two completely independent connected components A-B and C-D
# W matrix

W = np.array([
    [0, 1, 0, 0],
    [1, 0, 0, 0],
    [0, 0, 0, 2],
    [0, 0, 2, 0]
], dtype=float)

# D matrix

D = np.diag(W.sum(axis=1))

# L matrix

L = D - W

print("Laplacian matrix L:\n", L)

# eigenvalues and eigenvectors of L
eigvals, eigvecs = eigh(L)  # eigh is used for symmetric matrices
print("\nEigenvalues:", eigvals)

# eigenvectors
print("\nEigenvectors (columns):\n", np.round(eigvecs, 4))

# Check for zero eigenvalues
print("\nZero-eigenvalue eigenspace (spanned by first two eigenvectors):")
# all rows and the first two columns of eigvecs
print(np.round(eigvecs[:, :2], 4))

Laplacian matrix L:
 [[ 1. -1.  0.  0.]
 [-1.  1.  0.  0.]
 [ 0.  0.  2. -2.]
 [ 0.  0. -2.  2.]]

Eigenvalues: [0. 0. 2. 4.]

Eigenvectors (columns):
 [[-0.7071 -0.     -0.7071  0.    ]
 [-0.7071 -0.      0.7071  0.    ]
 [-0.     -0.7071  0.     -0.7071]
 [-0.     -0.7071  0.      0.7071]]

Zero-eigenvalue eigenspace (spanned by first two eigenvectors):
[[-0.7071 -0.    ]
 [-0.7071 -0.    ]
 [-0.     -0.7071]
 [-0.     -0.7071]]


the result is after normalization, we can also calculate by hand

In [15]:
import sympy as sp

# define L
L = sp.Matrix([[1, -1, 0, 0],
               [-1, 1, 0, 0],
               [0, 0, 2, -2],
               [0, 0, -2, 2]])

# The eigenvalue problem Lx = λx
# det(L−λI)=det(A−λI)⋅det(B−λI),
lam = sp.symbols('lam')
I = sp.eye(L.shape[0])  
det_poly = (L - lam*I).det()
print(sp.factor(det_poly))



# for egienvalue = 0, indicators of the connected components
# (1,1,0,0) and (0,0,1,1)

for lam in [0, 2, 4]:
    M = L - lam*I
    x = M.nullspace()   # returns the null space of M
    print(f"Eigenvectors for λ={lam}:")
    for vec in x:
        print(vec)      # vec is a 4*1 column vector
    print()


lam**2*(lam - 4)*(lam - 2)
Eigenvectors for λ=0:
Matrix([[1], [1], [0], [0]])
Matrix([[0], [0], [1], [1]])

Eigenvectors for λ=2:
Matrix([[-1], [1], [0], [0]])

Eigenvectors for λ=4:
Matrix([[0], [0], [-1], [1]])



## The normalized graph Laplacians

1. symmetric normalized Laplacian
2. random walk normalized Laplacian


# Graph cut point of view

1. RatioCut
2. normalized cut Ncut

In [23]:
import numpy as np

# adjacency matrix W
W = np.zeros((6,6))
# A completely connected graph weight 5
for i in [0,1,2]:
    for j in [0,1,2]:
        if i<j: W[i,j]=W[j,i]=5
# B completely connected graph weight 5
for i in [3,4,5]:
    for j in [3,4,5]:
        if i<j: W[i,j]=W[j,i]=5
# some weak connections between A and B weight 1
W[2,3]=W[3,2]=1  # (3,4)
W[1,4]=W[4,1]=1  # (2,5)

def cut_weight(W, A):
    A = set(A)
    B = set(range(W.shape[0])) - A
    return W[np.ix_(list(A), list(B))].sum()

def ratio_cut(W, parts):
    return sum(cut_weight(W, P)/len(P) for P in parts)

def normalized_cut(W, parts):
    D = np.diag(W.sum(axis=1))
    D_inv = np.linalg.inv(D)
    W_norm = D_inv @ W
    return sum(cut_weight(W_norm, P) for P in parts)

P1 = [ [0,1,2], [3,4,5] ]       # {1,2,3} vs {4,5,6}
P2 = [ [0], [1,2,3,4,5] ]       # {1} vs {2,3,4,5,6}

print("mincut(P1) =", cut_weight(W, P1[0]))  # 2.0 (if there are multiple cuts, return the half of sum)
print("mincut(P2) =", cut_weight(W, P2[0]))  # 10.0
print("RatioCut(P1) =", ratio_cut(W, P1))  # ~1.3333
print("RatioCut(P2) =", ratio_cut(W, P2))  # 12.0
print("Ncut(P1) =", normalized_cut(W, P1))  # ~0.3636
print("Ncut(P2) =", normalized_cut(W, P2))  # ~1.9091


mincut(P1) = 2.0
mincut(P2) = 10.0
RatioCut(P1) = 1.3333333333333333
RatioCut(P2) = 12.0
Ncut(P1) = 0.36363636363636365
Ncut(P2) = 1.9090909090909092
